In [ ]:
import pandas as pd

pd.set_option("display.max_rows", 15000)
pd.set_option("display.max_columns", 1000)
import boto3
import awswrangler as wr

boto3.setup_default_session(region_name="il-central-1")

In [ ]:
database = "winner-db"
odds = wr.athena.read_sql_query("SELECT * FROM api_odds", database=database)

In [ ]:
from api_request.api_scraper import generate_unique_id
odds.rename(columns={"id": "result_id"}, inplace=True)
odds["unique_id"] = odds.apply(generate_unique_id, axis=1)

In [ ]:
wr.s3.to_parquet(
    odds,
    path="s3://boaz-winner-api/bets",
    dataset=True,
    database=database,
    table="api-odds",
    partition_cols=["date_parsed", "type"],
    mode="overwrite_partitions",
)